# Mathematik für Biologiestudierende

Wintersemester 2025/26

28.01.2026

&copy; 2026 Prof. Dr. Rüdiger W. Braun 

# Wiederholung (interaktiv)

Gehen Sie auf die Website

* https://pingo.coactum.de

und geben Sie folgende Zugangsnummer ein

* **670719**

oder scannen Sie den QR-Code

![QR-Code](bilder/qr02.png)

# Themen

* Vorhersagen
* zwei Arten von Konfidenzintervallen bei Vorhersagen
* lineare Modelle mit kategoriellen Daten

In [ ]:
import numpy as np
np.set_printoptions(legacy='1.21')
import seaborn as sns
sns.set_theme()
sns.set_context('talk')
import pandas as pd
from scipy import stats
import statsmodels.formula.api as smf  #   <-----  neu

# Vorhersagen (prediction)

In [ ]:
galton = pd.read_csv('galton.csv') 
galton.describe()

In [ ]:
formel = 'childHeight ~ father + mother'

* In dieser Formel sind `father` und `mother` die erklärenden und `childHeight` die abhängige Variable
* Die erklärenden Variablen heißen auch *exogen*, die abhängigen *endogen*
* englisch:
  * abhängig: *dependent* oder *outcome variable*
  * erklärend:  *predictor*    

In [ ]:
modell = smf.ols(formel, galton)
res = modell.fit()
res.summary()

* Aufgabe:  Welche Größe erwarten wir für den Sohn eines 70" großen Vaters und einer 69.8" großen Mutter?

* In der Sprache der linearen Modelle handelt es sich um eine Vorhersage, engl. *prediction*

* Wir verwenden nun die Methode `get_prediction`
* Dazu müssen die Daten der erklärenden Variablen in einen DataFrame geschrieben werden

In [ ]:
anfrage = pd.DataFrame()
anfrage['father'] = [70]
anfrage['mother'] = [69.8]
#  rechte Seite ist immer ein array, auch wenn nur ein Wert berechnet werden soll
anfrage

In [ ]:
res.get_prediction(anfrage).summary_frame()

* Man erwartet, dass der Sohn 71.49" groß ist

* `mean`:  Wert, der im Mittel zu erwarten ist
* `mean_se`:  Standardabweichung für `mean`

* die vier anderen Werte sind untere bzw. obere Grenzen für Konfidenzintervalle
* `mean_ci` ist das Konfidenzintervall für den mittleren zu erwartenden Wert
* `obs_ci` ist das Konfidentintervall für den individuell zu beobachtenden Wert (engl: "observed")

# Konfidenzintervalle für den Mittelwert bzw. für die erwarteten Beobachtungen 

* `mean_ci_lower` und `mean_ci_upper` begrenzen das Konfidenzintervall für den Erwartungswert

* `obs_ci_lower` und `obs_ci_upper` begrenzen einen Bereich, der 95% der erwarteten Beobachtungen enthält

#### Beispiel:  Fische

* Fische werden gezüchtet.  In den ersten 24 Monaten wurden die folgenden Daten erhoben
* Diesen Daten werden benutzt, um das Wachstum der nächsten Generation zu prognostizieren

In [ ]:
fische = pd.read_csv('fische.csv')
fische.describe()

* Gewicht in g
* Höhe in mm

* Ein Züchter hat 1200 Fische in seinen Teichen, die alle gleichzeitig geschlüpft sind
* erste Frage:  Was ist das Konfidenzintervall für das Gesamtgewicht dieser Fische nach 18 Monaten zum Konfidenzniveau 95%?
* zweite Frage:  Wie muss das Netz gewählt werden, um nach 18 Monaten 97.5% der Fische zu fangen?

In [ ]:
formel1 = 'Gewicht ~ Monat'
modell1 = smf.ols(formel1, fische)

In [ ]:
res = modell1.fit()
res.summary()

In [ ]:
anfrage_fische = pd.DataFrame()
anfrage_fische['Monat'] = [18]
res.get_prediction(anfrage_fische).summary_frame()

* untere Vertrauensgrenze für das Gesamtgewicht von 1200 Fischen in kg:

In [ ]:
1200 * 1490.8 / 1000

Mit 97.5% Sicherheit werden mindestens 1789 kg Fisch geerntet

In [ ]:
formel2 = 'Höhe ~ Monat'
modell2 = smf.ols(formel2, fische)
res_hoehe = modell2.fit()
res_hoehe.summary()

In [ ]:
res_hoehe.get_prediction(anfrage_fische).summary_frame()

Um 97.5% der Fische zu fangen, muss das Netz so beschaffen sein, dass ein Fisch der Höhe 77.4mm nicht hindurch schlüpft

* Jetzt dasselbe nochmal für das Konfidenzniveau 0.99
* Achtung: `alpha` eingeben für Konfidenzniveau $1-\alpha$

In [ ]:
res_hoehe.get_prediction(anfrage_fische).summary_frame(alpha=0.01)

# Lineare Modelle mit kategoriellen Daten

In [ ]:
kinder = pd.read_csv('kinder.csv')
kinder.head()

In [ ]:
sns.lmplot(kinder, x='father', y='childHeight', hue='gender');

In [ ]:
formel = 'childHeight ~ father + mother + gender'
modell = smf.ols(formel, kinder)

In [ ]:
res = modell.fit()

In [ ]:
res.summary()

Hier wird eine Fallunterscheidung kodiert

$$
    \text{childHeight} = 16.5212 + 0.3928 \cdot \text{father} + 0.13176 \cdot \text{mother} + 
    \begin{cases}
        0.0 & \text{wenn Mädchen,} \\
        5.215 & \text{wenn Junge.}
    \end{cases}
$$

Die Terminologie kommt offenbar aus der Pharmazie:

* `female` ist hier der Standard (engl. "default")
* alles, was vom Standard abweicht, ist eine Behandlung (engl. "treatment")
* daher `T.male`
* was default und was treatment ist, entscheidet das Programm

## Prediction im kategoriellen Fall

In [ ]:
anfrage['gender'] = ['male']
anfrage

In [ ]:
res.get_prediction(anfrage).summary_frame()

* Vorhin war 71.49 herausgekommen
* Dies ist kein Rundungsfehler
* Es wird eine gemeinsame Steigung für alle Kinder berechnet

#### Beispiel Ratten

Wir kommen zu dem Rattenbeispiel aus Lektion 26 zurück:

* kontaminiertes Gelände: fange 10 Ratten
* unbelastetes Vergleichsgelände:  fange 10 Ratten
* für jede Ratte wird ihr Alter in Monaten und der Bleigehalt im Gewebe bestimmt

In [ ]:
ratten = pd.read_csv('ratten.csv')

In [ ]:
sns.lmplot(ratten, x='Alter', y='Belastung', hue='Gelände');

* Der t-Test zeigte keinen Unterschied zwischen den Ratten auf kontaminierten und nicht kontaminiertem Gelände.
* Die Ratten auf dem kontaminierten Gelände sind aber im Schnitt jünger.  
* Wir wollen 9 Monate alte Ratten vergleichen

In [ ]:
ratten['Differenz_zu_9'] = ratten.Alter - 9
ratten.head()

In [ ]:
formel = "Belastung ~ Differenz_zu_9 + Gelände"

In [ ]:
modell = smf.ols(formel, ratten)
res = modell.fit()

In [ ]:
res.summary()

* Der *p*-Wert von `Gelände[T.unkontaminiert]` ist 0.002
* Das ist der *p*-Wert des zweiseitigen Test auf Unterschiede des Bleigehalts bei 9 Monate alten Ratten

# Zweite Probeklausur

* Am Sonntag, dem 01.02.2026, werde ich eine zweite Probeklausur veröffentlichen
* Diese wird am 04.02.2026 besprochen

* Am 30.01.2026 erscheinen die letzten Übungsaufgaben
* Am 03.02.2026 findet die Vorlesung wie gewohnt statt